# Проект по SQL

## Описание проекта


Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных 

**Таблица books:**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors:**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers:**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Задания:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к БД

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
query_books = ''' SELECT *
                  FROM books;
              '''

In [4]:
books = pd.io.sql.read_sql(query_books, con = engine) 
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query_authors = ''' SELECT *
                    FROM authors;
                '''

In [6]:
authors = pd.io.sql.read_sql(query_authors, con = engine) 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query_publishers = ''' SELECT *
                       FROM publishers;
                   '''

In [8]:
publishers = pd.io.sql.read_sql(query_publishers, con = engine) 
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query_ratings = ''' SELECT *
                    FROM ratings;
                '''

In [10]:
ratings = pd.io.sql.read_sql(query_ratings, con = engine) 
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query_reviews = ''' SELECT *
                    FROM reviews;
                '''

In [12]:
reviews = pd.io.sql.read_sql(query_reviews, con = engine) 
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задачи
### Посчитайте, сколько книг вышло после 1 января 2000 года

In [30]:
s1 = '''SELECT COUNT(DISTINCT book_id)
           FROM books
           WHERE publication_date > '2000-01-01';
        '''

pd.read_sql(s1, con = engine)

,count
0,819


После 1 января вышло 819 книг

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [26]:
s2 = '''
SELECT books.book_id,
books.title, 
COUNT(DISTINCT reviews.review_id) AS count_review,  
AVG(ratings.rating) AS avg_ratings         
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id 
LEFT JOIN reviews ON reviews.book_id = books.book_id 
GROUP BY books.book_id
ORDER BY count_review DESC,avg_ratings DESC;
'''   

pd.read_sql(s2, con = engine)


,book_id,title,count_review,avg_ratings
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Максимальное количество обзоров у Twilight (Twilight #1) - Сумерки, 7 обзоров

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [31]:
s3 = '''
SELECT publishers.publisher,
COUNT(books.book_id) AS c_books
FROM publishers
LEFT JOIN books ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY publishers.publisher
ORDER BY c_books DESC
LIMIT 1
'''   

pd.read_sql(s3, con = engine)

,publisher,c_books
0,Penguin Books,42


Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books. Было выпущено 43 книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;


In [34]:
s4 = '''
SELECT authors.author,
AVG(subquery_ratings.avg_rating) AS avg_rating
FROM books
LEFT JOIN ( SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating_id) AS rating_cnt
FROM ratings
GROUP BY book_id) AS subquery_ratings ON subquery_ratings.book_id = books.book_id
LEFT JOIN authors ON authors.author_id = books.author_id
WHERE rating_cnt >= 50
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''

In [33]:
pd.io.sql.read_sql(s4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка книгу у автора J.K. Rowling

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
sql5 = '''
WITH av

AS (SELECT username, COUNT(review_id) as counter
FROM reviews AS r
WHERE username in (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50)
GROUP BY username)

SELECT AVG(counter)
FROM av
'''

pd.read_sql(sql5, con = engine)

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33 шт.

## Вывод:

В ходе данной работы мы изучили базу данных сервиса покупки книг по подписке.

По итогам проведения запросов к базе данных мы получили необходимую информацию:

- после 01.01.2000 г. было выпущено 821 книг.
- для каждой книги посчитали средняя оценка пользователей и количество обзоров
- больше всех книг (42 шт) выпустило издательство Penguin Books (объемом более 50 страниц)
- автор с самым высоким рейтингом книг - Джоан Роулинг
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33